Task 3 - Evaluating VGG Model on Out of Domain Data to Study Semantic Shift - CIFAR-100 Dataset

Making All the required Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchvision.models as models

Loading In the VGG Model and setting up the device to see if we have a valid GPU available

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.vgg19(pretrained=True)

c:\Program Files\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Program Files\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Getting The VGG Pretraind Model Ready for training. 

This step includes freezing all parameters in the feature extractor and replacing the classifier head with a new head based on the number of classes our dataset has which is 100 as CIFAR-100 dataset has 100 classes. We then define loss function and optimizer as. During this step we also move the model to our available device.

In [3]:
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.classifier[6].in_features  
model.classifier[6] = nn.Linear(num_ftrs, 100)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)  

In this step we define our Image transformation that we have to apply onto our CIFAR-100 Dataset. These transformations enhance the dataset and are helpful in Image classification. The outputed dataset with Image transformations applied is stored in the Variable trainset.

We also get the trainloader which essentially splits train_dataset into batches or chunks. This can be helpful in making the model train more faster and smoother

In [4]:
transform = transforms.Compose([
    transforms.Resize(224), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)

Files already downloaded and verified


Training and Finetuning the VGG Model

In this step the VGG model is trained for 3 epochs. Essentially we will be doing a forward pass, calculating the loss and then updating the weights in the backward pass.

In [5]:
model.train()
num_epochs = 3 

for epoch in range(num_epochs):
    running_loss = 0.0
    with tqdm(total=len(trainloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for i, (images, labels) in enumerate(trainloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad() 

            outputs = model(images)
            loss = criterion(outputs, labels) 

            loss.backward() 
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1)) 
            pbar.update(1)

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}], Step [{i + 1}], Loss: {running_loss / (i + 1):.4f}')

save_path = "models/vgg_19_CIFAR_100_TASK3.pth"
torch.save(model.state_dict(), save_path) 
print(f'Model saved to {save_path}')

Epoch 1/3:   6%|▋         | 100/1563 [00:45<04:45,  5.13batch/s, loss=2.95]

Epoch [1], Step [100], Loss: 2.9493


Epoch 1/3:  13%|█▎        | 200/1563 [01:05<04:44,  4.78batch/s, loss=2.52]

Epoch [1], Step [200], Loss: 2.5197


Epoch 1/3:  19%|█▉        | 300/1563 [01:26<04:59,  4.21batch/s, loss=2.35]

...

Epoch [3], Step [1400], Loss: 1.7238


Epoch 3/3:  96%|█████████▌| 1500/1563 [08:01<00:15,  4.13batch/s, loss=1.73]

Epoch [3], Step [1500], Loss: 1.7312


Epoch 3/3: 100%|██████████| 1563/1563 [08:16<00:00,  3.15batch/s, loss=1.73]


Model saved to models/vgg_19_CIFAR_100_TASK3.pth


Running the Testing Loop

Evaluate the VGG model. See how Well the Model performs on an Out of Domain Dataset like PACS. It Represents A semantic Shift so a drop in Accuracy is expected compared to CIFAR-10 and PACS dataset accuracy.

In [6]:
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=4)

model.eval()

correct = 0
total = 0

with torch.no_grad(): 
    for images, labels in tqdm(testloader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on CIFAR-100 test set: {accuracy:.2f}%')


Files already downloaded and verified


Evaluating: 100%|██████████| 313/313 [01:24<00:00,  3.73it/s]

Accuracy on CIFAR-100 test set: 60.01%
